# Async (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installing Packages Expecto........

Installed Packages Expecto, 10.1.0

In [ ]:
#!import ../nbs/Common.fs

In [ ]:
open Common

## runWithTimeout

In [ ]:
let runWithTimeout (timeout : int) fn =
    let getLocals () = $"timeout: {timeout} / {getLocals ()}"

    let timeoutTask = async {
        do! Async.Sleep timeout
        return None, getLocals
    }
    
    let task = async {
        try
            return Async.RunSynchronously (fn, timeout) |> Some, getLocals
        with
        | :? System.TimeoutException as ex ->
            let getLocals () = $"exception: {ex.Message} / {getLocals ()}"
            return None, getLocals
        | e -> return raise e
    }

    [| timeoutTask; task |]
    |> Array.map Async.StartAsTask
    |> System.Threading.Tasks.Task.WhenAny
    |> fun task ->
        match task.Result.Result with
        | None, getLocals ->
            trace Debug (fun () -> "runWithTimeout") getLocals
            None
        | result, _ -> result

In [ ]:
//// test

Async.Sleep 50
|> runWithTimeout 10
|> _equal None

16:14:17 #1 [Debug] runWithTimeout / timeout: 10
<null>


In [ ]:
//// test

Async.Sleep 10
|> runWithTimeout 50
|> _equal (Some ())

FSharpOption<Unit>
      Value: <null>
